## Benchmarking

In [1]:
import ROOT
import os
RDataFrame = ROOT.RDF.Experimental.Distributed.OSCAR.RDataFrame

oscarclient = {
    "minio_endpoint": os.environ['minio_endpoint'],
    "minio_access":   os.environ['minio_access'],
    "minio_secret":   os.environ['minio_secret'],
    "oscar_endpoint": os.environ['oscar_endpoint'],
    "oscar_access":   os.environ['oscar_access'],
    "oscar_secret":   os.environ['oscar_secret']
}

cern_data = "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"

Welcome to JupyROOT 6.27/01


In [2]:
# Define funcion.
def dimuon_analysis(df):
    # For simplicity, select only events with exactly two muons and require opposite charge
    df_2mu = df.Filter("nMuon == 2", "Events with exactly two muons")
    df_os = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]", "Muons with opposite charge")
    
    # Compute invariant mass of the dimuon system
    df_mass = df_os.Define("Dimuon_mass", "InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")
    
    # Make histogram of dimuon mass spectrum. Note how we can set titles and axis labels in one go.
    h = df_mass.Histo1D(("Dimuon_mass", "Dimuon mass;m_{#mu#mu} (GeV);N_{Events}", 30000, 0.25, 300), "Dimuon_mass")
    
    # Produce plot
    ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
    c = ROOT.TCanvas("c", "", 800, 700)
    c.SetLogx(); c.SetLogy()
    
    h.SetTitle("")
    h.GetXaxis().SetTitleSize(0.04)
    h.GetYaxis().SetTitleSize(0.04)
    h.Draw()
    
    label = ROOT.TLatex(); label.SetNDC(True)
    label.DrawLatex(0.175, 0.740, "#eta")
    label.DrawLatex(0.205, 0.775, "#rho,#omega")
    label.DrawLatex(0.270, 0.740, "#phi")
    label.DrawLatex(0.400, 0.800, "J/#psi")
    label.DrawLatex(0.415, 0.670, "#psi'")
    label.DrawLatex(0.485, 0.700, "Y(1,2,3S)")
    label.DrawLatex(0.755, 0.680, "Z")
    label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
    label.SetTextSize(0.030); label.DrawLatex(0.630, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")
    
    c.SaveAs(f"dimuon_spectrum.pdf")


# Create RDataFrame
df = RDataFrame("Events", cern_data, 
                oscarclient = oscarclient,
                npartitions = 16)

# Execute defined analysis
dimuon_analysis(df)

Bucket name: default-88677a63-f997-4bfa-b408-9395a8566b84
Bucket does not exist. Trying to create it.
Creating bucket...
Bucket created!
Creating services...
Creating mapper service for bucket default-88677a63-f997-4bfa-b408-9395a8566b84
Trigger path: default-88677a63-f997-4bfa-b408-9395a8566b84/mapper-jobs
Write prefixes: ['partial-results']
Creating reducer service for bucket default-88677a63-f997-4bfa-b408-9395a8566b84
Trigger path: default-88677a63-f997-4bfa-b408-9395a8566b84/partial-results
Write prefixes: ['partial-results']
Done creating services!
Done setting up OSCAR backend!
<Response [201]>
<Response [201]>
Starting timer
Target Name: 0_15
File 8_8 written to partial-results folder.
File 12_12 written to partial-results folder.
File 6_6 written to partial-results folder.
File 9_9 written to partial-results folder.
File 14_14 written to partial-results folder.
File 10_10 written to partial-results folder.
File 3_3 written to partial-results folder.
File 4_4 written to partial

Info in <TCanvas::Print>: pdf file dimuon_spectrum.pdf has been created
